In [1]:
import glob
import os

from astropy.io import fits
import numpy as np
import numpy.ma as ma
import pandas as pd

In [2]:
working_directory = '/grp/hst/wfc3u/final_postflash_2021/yearly_reference/'
today = '09_05_2022'
postflash_data = pd.read_pickle('/grp/hst/wfc3u/final_postflash_2021/notebooks/Feb_2022_flc_all_stats_postflash.pkl')

In [3]:
postflash_data['datetime']

postflash_data=postflash_data.sort_values(by='datetime')
print(postflash_data)

                     filename  propid  \
rootname                                
ic6da5qoq  ic6da5qoq_flc.fits   13078   
ic6da5qpq  ic6da5qpq_flc.fits   13078   
ic6da5qqq  ic6da5qqq_flc.fits   13078   
ic6da5qsq  ic6da5qsq_flc.fits   13078   
ic6da5qtq  ic6da5qtq_flc.fits   13078   
...                       ...     ...   
ielu15jeq  ielu15jeq_flc.fits   16570   
ielu13jzq  ielu13jzq_flc.fits   16570   
ielu13k0q  ielu13k0q_flc.fits   16570   
ielu13k2q  ielu13k2q_flc.fits   16570   
ielu13k3q  ielu13k3q_flc.fits   16570   

                                                        path     obsdate  \
rootname                                                                   
ic6da5qoq  /grp/hst/wfc3u/final_postflash_2021/mastDownlo...  2012-11-14   
ic6da5qpq  /grp/hst/wfc3u/final_postflash_2021/mastDownlo...  2012-11-14   
ic6da5qqq  /grp/hst/wfc3u/final_postflash_2021/mastDownlo...  2012-11-14   
ic6da5qsq  /grp/hst/wfc3u/final_postflash_2021/mastDownlo...  2012-11-14   
ic6da5qtq 

In [4]:
fullframe_pf = postflash_data.loc[(postflash_data['subarray'] == False)] 
fullframe_pf_A = postflash_data.loc[(postflash_data['subarray'] == False) & (postflash_data['shutter'] == 'A') & (postflash_data['flash_cur'] == 'MED') & (postflash_data['flash_dur'] == 100.0)] 

In [5]:
def stack(list_of_files,out_file,error_file):
    hdr = fits.getheader(list_of_files[0], 1)
    nx = hdr['NAXIS1']
    ny = hdr['NAXIS2']
    nf = len(list_of_files)
    # Setting up the empty data, rms, and error arrays and getting the data
    data_array_1 = np.empty((nf, ny, nx), dtype=float)
    data_array_2 = np.empty((nf, ny, nx), dtype=float)
    set_data=fits.getdata(list_of_files[0], 1)
    rms_1 = np.zeros(len(list_of_files), dtype=float)
    rms_2 = np.zeros(len(list_of_files), dtype=float)
    error_array_1 = np.empty((nf, ny, nx), dtype=float)
    error_array_2 = np.empty((nf, ny, nx), dtype=float)
    total_error_1 = np.zeros_like(set_data, dtype=float)
    total_error_2 = np.zeros_like(set_data, dtype=float)
    for i , f in enumerate(list_of_files):
        data_1=fits.getdata(f, 1)
        data_2=fits.getdata(f, 4)
        error_1=fits.getdata(f, 2)
        error_2=fits.getdata(f, 5)
        DQ_1=fits.getdata(f, 3)
        DQ_2=fits.getdata(f, 6)
        #data_1=data1
        #data_2=data2
        mask_1=np.zeros_like(data_1, dtype=bool)
        mask_2=np.zeros_like(data_2, dtype=bool)
        mask_1[DQ_1>=2**13]= True
        mask_2[DQ_2>=2**13]= True
        error_1[(0<DQ_1 & (DQ_1<2**13))]=0.00001
        error_2[(0<DQ_2& (DQ_2<2**13))]=0.00001
        error_1_sq=error_1**2
        error_2_sq=error_2**2
        masked_data_1= ma.array(data=data_1, mask=mask_1)
        masked_data_2= ma.array(data=data_2, mask=mask_2)
        data_array_1[i, :, :] = masked_data_1
        rms_1[i] = masked_data_1.std()
        data_array_2[i, :, :] = masked_data_2
        rms_2[i] = masked_data_2.std()
        total_error_1=total_error_1+(error_1_sq)
        total_error_2=total_error_2+(error_2_sq)
    sr_total_error_1=np.sqrt(total_error_1)
    sr_total_error_2=np.sqrt(total_error_2)
    fin_error_1=(sr_total_error_1/(float(len(list_of_files))))
    fin_error_2=(sr_total_error_2/(float(len(list_of_files))))
    image_median_1 = np.median(data_array_1, axis=0)
    image_median_2 = np.median(data_array_2, axis=0)
    new_hdul = fits.HDUList()
    new_hdul.append(fits.ImageHDU(image_median_1))
    new_hdul.append(fits.ImageHDU(image_median_2))
    new_hdul.writeto(out_file, overwrite=True)
    #error
    new_hdul = fits.HDUList()
    new_hdul.append(fits.ImageHDU(fin_error_1))
    new_hdul.append(fits.ImageHDU(fin_error_2))
    new_hdul.writeto(error_file,overwrite=True)

To run the stacking code you need to run: stack(path_list, path_outfile, path_error_outfile)

path_list - this is the list you create from the path column of the subset of the pandas database. 

path_outfile - this is the path and filename you want for your outfile. If you just put "outfile.fits" the file will save where you are running the notebook. I suggest start with the path to wfc3u so we keep them in a central location and to have the file be descriptive. Ex: "/grp/hst/wfc3u/postflash_2021/2020_stack_flt.fits"

path_error_outfile - this is the path and filename you want for your outfile of the calcuated error. If you just put "outfile.fits" the file will save where you are running the notebook. I suggest start with the path to wfc3u so we keep them in a central location and to have the file be descriptive. Ex: "/grp/hst/wfc3u/postflash_2021/2020_stack_error_flt.fits"


Below I show how to create the proper cut for a single year of data from the pandas database which has already been cut to Fullframe, Medium current, Shutter A. 

From there I update the path_list, outfile, and error_outfile to be specfic to the year being tested. 

Once that is run, I use those parameters to run the `stack` function. Outputs will be saved to '/grp/hst/wfc3u/postflash_2021/'. 

I then update a copy of a new set to 2013 and run `stack` on that set. 

The following years/groups of years can be done the same way. Following the creation of all the stacked files we will still need to calculate the mean/median of the stacked images to plot them over time.

In [6]:
fullframe_pf_A_2012 = fullframe_pf_A[(fullframe_pf_A['datetime'] > '2012-01-01 00:00:00') & (fullframe_pf_A['datetime'] < '2013-11-14 00:00:00')]
paths_2012 = fullframe_pf_A_2012.path.tolist()
print(len(paths_2012))
outfile_2012 = '{}2012_fullframe_A_flc_stack_{}.fits'.format(working_directory, today)
error_outfile_2012= '{}2012_fullframe_A_flc_error_stack_{}.fits'.format(working_directory, today)

28


In [7]:
stack(paths_2012, outfile_2012, error_outfile_2012)

In [8]:
fullframe_pf_A_2013 = fullframe_pf_A[(fullframe_pf_A['datetime'] > '2013-01-01 00:00:00') & (fullframe_pf_A['datetime'] < '2014-01-01 00:00:00')]
paths_2013 = fullframe_pf_A_2013.path.tolist()
print(len(paths_2013))
outfile_2013 = '{}2013_fullframe_A_flc_stack_{}.fits'.format(working_directory, today)
error_outfile_2013= '{}2013_fullframe_A_flc_error_stack_{}.fits'.format(working_directory, today)

32


In [9]:
stack(paths_2013, outfile_2013, error_outfile_2013)

In [10]:
fullframe_pf_A_2014 = fullframe_pf_A[(fullframe_pf_A['datetime'] > '2014-01-01 00:00:00') & (fullframe_pf_A['datetime'] < '2015-01-01 00:00:00')]
paths_2014 = fullframe_pf_A_2014.path.tolist()
print(len(paths_2014))
outfile_2014 = '{}2014_fullframe_A_flc_stack_{}.fits'.format(working_directory, today)
error_outfile_2014= '{}2014_fullframe_A_flc_error_stack_{}.fits'.format(working_directory, today)

30


In [11]:
stack(paths_2014, outfile_2014, error_outfile_2014)

In [12]:
fullframe_pf_A_2015 = fullframe_pf_A[(fullframe_pf_A['datetime'] > '2015-01-01 00:00:00') & (fullframe_pf_A['datetime'] < '2016-01-01 00:00:00')]
paths_2015 = fullframe_pf_A_2015.path.tolist()
print(len(paths_2015))
outfile_2015 = '{}2015_fullframe_A_flc_stack_{}.fits'.format(working_directory, today)
error_outfile_2015 = '{}2015_fullframe_A_flc_error_stack_{}.fits'.format(working_directory, today)

35


In [13]:
stack(paths_2015, outfile_2015, error_outfile_2015)

In [14]:
fullframe_pf_A_2016 = fullframe_pf_A[(fullframe_pf_A['datetime'] > '2016-01-01 00:00:00') & (fullframe_pf_A['datetime'] < '2017-01-01 00:00:00')]
paths_2016 = fullframe_pf_A_2016.path.tolist()
print(len(paths_2016))
outfile_2016 = '{}2016_fullframe_A_flc_stack_{}.fits'.format(working_directory, today)
error_outfile_2016 = '{}2016_fullframe_A_flc_error_stack_{}.fits'.format(working_directory, today)

35


In [15]:
stack(paths_2016, outfile_2016, error_outfile_2016)

In [16]:
fullframe_pf_A_2017 = fullframe_pf_A[(fullframe_pf_A['datetime'] > '2017-01-01 00:00:00') & (fullframe_pf_A['datetime'] < '2018-01-01 00:00:00')]
paths_2017 = fullframe_pf_A_2017.path.tolist()
print(len(paths_2017))
outfile_2017 = '{}2017_fullframe_A_flc_stack_{}.fits'.format(working_directory, today)
error_outfile_2017 = '{}2017_fullframe_A_flc_error_stack_{}.fits'.format(working_directory, today)

35


In [17]:
stack(paths_2017, outfile_2017, error_outfile_2017)

In [18]:
fullframe_pf_A_2018 = fullframe_pf_A[(fullframe_pf_A['datetime'] > '2018-01-01 00:00:00') & (fullframe_pf_A['datetime'] < '2019-01-01 00:00:00')]
paths_2018 = fullframe_pf_A_2018.path.tolist()
print(len(paths_2018))
outfile_2018 = '{}2018_fullframe_A_flc_stack_{}.fits'.format(working_directory, today)
error_outfile_2018 = '{}2018_fullframe_A_flc_error_stack_{}.fits'.format(working_directory, today)

35


In [19]:
stack(paths_2018, outfile_2018, error_outfile_2018)

In [20]:
fullframe_pf_A_2019 = fullframe_pf_A[(fullframe_pf_A['datetime'] > '2019-01-01 00:00:00') & (fullframe_pf_A['datetime'] < '2020-01-01 00:00:00')]
paths_2019 = fullframe_pf_A_2019.path.tolist()
print(len(paths_2019))
outfile_2019 = '{}2019_fullframe_A_flc_stack_{}.fits'.format(working_directory, today)
error_outfile_2019 = '{}2019_fullframe_A_flc_error_stack_{}.fits'.format(working_directory, today)

35


In [21]:
stack(paths_2019, outfile_2019, error_outfile_2019)

In [22]:
fullframe_pf_A_2020 = fullframe_pf_A[(fullframe_pf_A['datetime'] > '2020-01-01 00:00:00') & (fullframe_pf_A['datetime'] < '2021-01-01 00:00:00')]
paths_2020 = fullframe_pf_A_2020.path.tolist()
print(len(paths_2020))
outfile_2020 = '{}2020_fullframe_A_flc_stack_{}.fits'.format(working_directory, today)
error_outfile_2020 = '{}2020_fullframe_A_flc_error_stack_{}.fits'.format(working_directory, today)

38


In [23]:
stack(paths_2020, outfile_2020, error_outfile_2020)

In [24]:
fullframe_pf_A_2021 = fullframe_pf_A[(fullframe_pf_A['datetime'] > '2021-01-01 00:00:00') & (fullframe_pf_A['datetime'] < '2022-01-01 00:00:00')]
paths_2021 = fullframe_pf_A_2021.path.tolist()
print(len(paths_2021))
outfile_2021 = '{}2021_fullframe_A_flc_stack_{}.fits'.format(working_directory, today)
error_outfile_2021 = '{}2021_fullframe_A_flc_error_stack_{}.fits'.format(working_directory, today)

31


In [25]:
stack(paths_2021, outfile_2021, error_outfile_2021)

Adding a change_permissions function: 

In [26]:
def change_permissions(path_to_files):
    """Change permissions of the output files and directories. 
        Code structure borrowed from cal_uvis_make_darks/FileIO.py. 
        Parameters
        ----------
            path_to_files : str
                Path to the directory that contains the files you want
                to update. 
    """
    
    os.chdir(path_to_files)
    all_directories = glob.glob('*')

    for directory in all_directories:
        try: 
            os.chmod(directory, 0o775)
        except:
            print('Can not update. Onto Next.')
        for root, subdirs, files in os.walk(directory):
            for subdir in subdirs:
                try:
                    os.chmod(os.path.join(root, subdir), 0o775)
                except:
                    print('Can not update. Onto Next.')
            for name in files:
                try:
                    os.chmod(os.path.join(root, name), 0o775)
                except:
                    print('Can not update. Onto Next.')

In [27]:
change_permissions(working_directory)